<a href="https://colab.research.google.com/github/pulkitpathak99/IMDB-Sentiment-Analysis-using-LSTM/blob/main/IMDB_Sentiment_Analysis_using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install kaggle


**Dependency** **Installation**

In [49]:
import os
import json # to access the configurational file

from zipfile import ZipFile #to extract kaggle dataset file
import pandas as pd #loading dataset csv into pandas dataframe
from sklearn.model_selection import train_test_split #to split dataset into training and testing set
from tensorflow.keras.models import Sequential #keras provide API which makes it easier to create neural networks , we'll use layers like embedding
from tensorflow.keras.layers import Input, Dense, Embedding , LSTM # dense layer -> fully connected layer , embedding layer -> first layer and then LSTM
from tensorflow.keras.preprocessing.text import Tokenizer #maps words to certain integers
from tensorflow.keras.preprocessing.sequence import pad_sequences #check if all input data is in same shape

**Collecting the data -> Kaggle API**

In [4]:
kaggle_dict= json.load(open("kaggle.json"))



In [6]:
#set up kaggle API key as environment variables as i don't want to risk my kaggle credentials in my code
os.environ["KAGGLE_USERNAME"]= kaggle_dict["username"]
os.environ["KAGGLE_KEY"]= kaggle_dict["key"]


In [7]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
 97% 25.0M/25.7M [00:00<00:00, 131MB/s] 
100% 25.7M/25.7M [00:00<00:00, 122MB/s]


In [8]:
!ls

imdb-dataset-of-50k-movie-reviews.zip  kaggle.json  sample_data


In [11]:
#unzip the file
with ZipFile('imdb-dataset-of-50k-movie-reviews.zip') as zip_ref:
  zip_ref.extractall()

In [12]:
!ls

'IMDB Dataset.csv'   imdb-dataset-of-50k-movie-reviews.zip   kaggle.json   sample_data


In [13]:
data=pd.read_csv('/content/IMDB Dataset.csv')

In [14]:
data.shape #prints row and col

(50000, 2)

In [16]:
data.head() #prints first 5 rows gives idea about the data


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [17]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [18]:
data["sentiment"].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [19]:
data.replace({"sentiment" : {"positive":1 , "negative": 0}}, inplace=True) #positive is now changed inplacely to 1 while negative is to 0

In [20]:
data["sentiment"].value_counts()

,count
sentiment,
1,25000
0,25000


In [23]:
#split data into training and testing
train_data, test_data= train_test_split(data, test_size=0.2, random_state=42)
print(train_data.shape)
test_data.shape

(40000, 2)


(10000, 2)

**Data Preprocessing**

In [27]:
tokenizer= Tokenizer(num_words=5000) #used to convert the words to integer
tokenizer.fit_on_texts(train_data["review"])
X_train=pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200) #Words are mapped based on their frequency in the dataset
X_test=pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

In [30]:
print(X_train)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [31]:
print(X_test)

[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]


In [32]:
Y_train= train_data["sentiment"]
Y_test= test_data["sentiment"]

In [35]:
print(Y_train)

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64


In [34]:
print(Y_test)

33553    1
9427     1
199      0
12447    1
39489    0
        ..
28567    0
25079    1
18707    1
15200    0
5857     1
Name: sentiment, Length: 10000, dtype: int64


**LSTM - Long Short Term Memory**

In [50]:
#In LSTM, when provided with input the model should not look for all the data to provide output instead it should have memory that it is aware of the trend
# model building
model = Sequential()
model.add(Input(shape=(200,)))
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid"))

model.summary()



/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_7 (Embedding)              │ (None, 200, 128)            │         640,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_7 (LSTM)                        │ (None, 128)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 771,713 (2.94 MB)

 Trainable params: 771,713 (2.94 MB)

 Non-trainable params: 0 (0.00 B)

In [52]:
#compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [54]:
#training the model
model.fit(X_train, Y_train, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 174s 335ms/step - accuracy: 0.7287 - loss: 0.5331 - val_accuracy: 0.8238 - val_loss: 0.4054
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 170s 341ms/step - accuracy: 0.8587 - loss: 0.3416 - val_accuracy: 0.8303 - val_loss: 0.3725
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 171s 342ms/step - accuracy: 0.8719 - loss: 0.3139 - val_accuracy: 0.8687 - val_loss: 0.3251
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 183s 305ms/step - accuracy: 0.8982 - loss: 0.2597 - val_accuracy: 0.8583 - val_loss: 0.3473
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 200s 301ms/step - accuracy: 0.9076 - loss: 0.2400 - val_accuracy: 0.8786 - val_loss: 0.3148


**Model Evaluation**

In [55]:
loss,accuracy= model.evaluate(X_test, Y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 38s 121ms/step - accuracy: 0.8859 - loss: 0.3030


**Building it into predictive system**

In [59]:
def predictive_sentiment(review):
  #tokenize and pad review
  sequence= tokenizer.texts_to_sequences([review])
  padded_sequence= pad_sequences(sequence, maxlen=200)
  prediction= model.predict(padded_sequence)
  sentiment= "positive" if prediction[0][0]>0.5 else "negative"
  return sentiment

In [60]:
new_review= "The movie sucked ass!"
sentiment= predictive_sentiment(new_review)
print(f"The Sentiment for this review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 287ms/step
The Sentiment for this review is: negative


In [64]:
new_review= "The movie was ass!"
sentiment= predictive_sentiment(new_review)
print(f"The Sentiment for this review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
The Sentiment for this review is: negative


In [67]:
new_review= "I can't believe i sat 2 hours for this so-called masterpiece which it was not!"
sentiment= predictive_sentiment(new_review)
print(f"The Sentiment for this review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
The Sentiment for this review is: negative
